In [1]:
from __future__ import print_function
import keras
import pickle as pk
import numpy as np
import keras.layers as kl
import keras.backend as K
import tensorflow as tf
K.set_image_data_format('channels_first')
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from Gradient_Reverse_Layer import GradientReversal

Using TensorFlow backend.


In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import gzip


import sys
if (sys.version_info > (3, 0)):
    import pickle as pkl
else: #Python 2.7 imports
    import cPickle as pkl

import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, concatenate
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D
from keras.regularizers import Regularizer
from keras.preprocessing import sequence
import time

In [ ]:
class DANN(object):
    def __init__(self, width=28, height = 28, channels = 1, classes=1, features=1, batch_size=1, grl='auto', summary=False, model_plot=False):
        ## Set Defualts
        self.learning_phase = K.variable(1)
        self.domain_invariant_features = None
        self.width = width
        self.height = 2
        self.channels = channels
        self.input_shape = (channels, width)
        self.classes = classes
        self.features = features
        self.batch_size = batch_size
        self.grl = 'auto'
        # Set reversal gradient value.
        if grl is 'auto':
            self.grl_rate = 1.0
        else:
            self.grl_rate = grl
        self.summary = summary
        self.model_plot = model_plot

        # Build the model
        self.model = self._build()
        
        # Print and Save the model summary if requested.
        if self.summary:
            self.model.summary()
        if self.model_plot:
            plot_model(self.model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

    def feature_extractor(self, inp):
        ''' 
        This function defines the structure of the feature extractor part.
        '''
        out = kl.Conv2D(filters=32, kernel_size=(3, 3), padding="same",
                        activation="relu", data_format='channels_first')(inp)
        out = kl.MaxPooling2D(pool_size=(2, 2))(out)
        
        nb_filter = 100
        filter_length = 3
#         out = Convolution1D(filters=nb_filter,
#                        kernel_size=filter_length,
#                        padding='same',
#                        activation='relu',
#                        strides=1)(inp)
#         out = GlobalMaxPooling1D()(out)


        out = kl.Dropout(0.5)(out)
#         out = kl.Flatten()(out)


        feature_output = kl.Dense(self.features, activation="relu")(out)
        self.domain_invariant_features = feature_output
        return feature_output

    def classifier(self, inp):
        ''' 
        This function defines the structure of the classifier part.
        '''
#         out = kl.Dense(128, activation="relu")(inp)
#         out = kl.Dropout(0.5)(out)
        classifier_output = kl.Dense(self.classes, activation="softmax", name="classifier_output")(inp)
        return classifier_output

    def discriminator(self, inp):
        ''' 
        This function defines the structure of the discriminator part.
        '''
        out = kl.Dense(128, activation="relu")(inp)
#         out = kl.Dropout(0.5)(out)
        discriminator_output = kl.Dense(2, activation="softmax", name="discriminator_output")(out)
        return discriminator_output

    def _build(self):
        '''
        This function builds the network based on the Feature Extractor, Classifier and Discriminator parts.
        '''
#        BEGIN - UBAH INPUT

        words_input = Input(shape=self.input_shape, dtype='int32', name='words_input')
        words = Embedding(embeddings.shape[0], embeddings.shape[1], weights=[embeddings], trainable=False)(words_input)

        distance1_input = Input(shape=self.input_shape, dtype='int32', name='distance1_input')
        distance1 = Embedding(max_position, position_dims)(distance1_input)

        distance2_input = Input(shape=self.input_shape, dtype='int32', name='distance2_input')
        distance2 = Embedding(max_position, position_dims)(distance2_input)
        print('77777777777777777777777')
        print(words_input.shape)
        
#         CUBO RESHAPE, GAGAL KARENA INPUT LAYER BERUBAH
#         s = tf.shape(words_input)
#         print(s)
#         words_input = tf.reshape(words_input, [-1, self.input_shape[0], self.input_shape[1], 300])
#         print(words_input.shape)
# #         s = tf.shape(a)
#         c = tf.reshape(b, shape=[s[0], s[1], 3])
#         words_input = np.expand_dims(words_input, axis=3)
#         words_input = np.tile(trainX, [1, 1, 1, 300])
        
#         distance1_input = np.expand_dims(distance1_input, axis=3)
#         distance1_input = np.tile(trainX, [1, 1, 1, 50])
        
#         distance2_input = np.expand_dims(distance2_input, axis=3)
#         distance2_input = np.tile(trainX, [1, 1, 1, 50])
        

        inp = concatenate([words, distance1, distance2])
        inp = [words, distance1, distance2]
        input_array = [words_input, distance1_input, distance2_input]

#        END UBAH INPUT

        import time
        print('000000000000000000000000000')
        print(inp[0].shape)
#         BEGIN CNN
        output = concatenate([words, distance1, distance2])
        
#         output = tf.reshape(output, [-1, self.input_shape[0], self.input_shape[1], 400])
#         print(output.shape)
        
        feature_output = self.feature_extractor(output)
#         END CNN
        
        self.grl_layer = GradientReversal(1.0)
        feature_output_grl = self.grl_layer(feature_output)
        labeled_feature_output = kl.Lambda(lambda x: K.switch(K.learning_phase(), K.concatenate([x[:int(self.batch_size//2)], x[:int(self.batch_size//2)]], axis=0), x), output_shape=lambda x: x[0:])(feature_output_grl)

        classifier_output = self.classifier(labeled_feature_output)
        discriminator_output = self.discriminator(feature_output)
        
        print('111111111111111111')
        print(input_array[0].shape)
        model = keras.models.Model(inputs=input_array, outputs=[discriminator_output, classifier_output])
        return model

    def batch_generator(self, trainX, trainY=None, batch_size=1, shuffle=True):
        '''
        This function generates batches for the training purposes.
        '''
        print('trainX.shape[0]')
        print(trainX.shape[0])
        print('44444444444444444444444')
        print('batch_size, self.channels, self.width, self.height')
        print(batch_size, self.channels, self.width, self.height)


        if shuffle:
            index = np.random.randint(0, len(trainX) - batch_size)
        else:
            index = np.arange(0, len(trainX), batch_size)
        while trainX.shape[0] > index + batch_size:
            batch = trainX[index : index + batch_size]
            batch = batch.reshape(batch_size, self.width, self.height, self.channels)
            if trainY is not None:
                batch_labels = trainY[index : index + batch_size]
                yield batch, batch_labels
            else:
                yield batch
            index += batch_size

    def compile(self, optimizer):
        '''
        This function compiles the model based on the given optimization method and its parameters.
        '''
        self.model.compile(optimizer=optimizer, loss={'classifier_output': 'binary_crossentropy', 'discriminator_output': 'binary_crossentropy'}, loss_weights={'classifier_output': 0.5, 'discriminator_output': 1.0})

    def train(self, trainX, trainDX, trainY=None, epochs=1, batch_size=1, verbose=True, save_model=None):
        '''
        This function trains the model using the input and target data, and saves the model if specified.
        '''
        print('222222222222222')
        print(trainX.shape)
        # Prepare batch 
        for cnt in range(epochs):
#             data for the model training.
            Labeled = self.batch_generator(trainX, trainY, batch_size=batch_size // 2)
            UNLabeled = self.batch_generator(trainDX, batch_size=batch_size // 2)
            
            # Settings for learning rate.
            p = np.float(cnt) / epochs
            lr = 0.01 / (1. + 10 * p)**0.75

            # Settings for reverse gradient magnitude (if it's set to be automatically calculated, otherwise set by user.)
            if self.grl is 'auto':
                self.grl_layer.l = 2. / (1. + np.exp(-10. * p)) - 1

            # Re-compile model to adopt new learning rate and gradient reversal value.
            self.compile(keras.optimizers.SGD(lr))
            
            # Loop over each batch and train the model.
            for batchX, batchY in Labeled:
                # Get the batch for unlabeled data. If the batches are finished, regenerate the batches agian.
                try:
                    batchDX = next(UNLabeled)
                except:
                    UNLabeled = self.batch_generator(trainDX, batch_size=batch_size // 2)
                # Combine the labeled and unlabeled images along with the discriminative results.
                combined_batchX = np.concatenate((batchX, batchDX))
                batch2Y = np.concatenate((batchY, batchY))
                combined_batchY = np.concatenate((np.tile([0, 1], [batchX.shape[0], 1]), np.tile([1, 0], [batchDX.shape[0], 1])))
                # Train the model
                
                metrics = self.model.train_on_batch({'words_input': combined_batchX,
                                                    'distance1_input': combined_batchX,
                                                    'distance2_input': combined_batchX},
                                                    {'classifier_output': batch2Y, 'discriminator_output':combined_batchY})
            # Print the losses if asked for.
            if verbose:
                print("Epoch {}/{}\n\t[Generator_loss: {:.4}, Discriminator_loss: {:.4}, Classifier_loss: {:.4}]".format(cnt+1, epochs, metrics[0], metrics[1], metrics[2]))
        # Save the model if asked for.
        if save_model is not None and isinstance(save_model, str):
            if save_model[-3:] is not ".h5":
                save_model = ''.join((save_model, ".h5"))
            self.model.save(save_model)
        elif save_model is not None and not isinstance(save_model, str):
            raise TypeError("The input must be a filename for model settings in string format.")


    def evaluate(self, testX, testY=None, weight_loc=None, save_pred=None, verbose=False):
        '''
        This function evaluates the model, and generates the predicted classes.
        '''
        if weight_loc is not None:
            self.compile(keras.optimizers.SGD())
            self.model.load_weights(weight_loc)
        _, yhat_class = self.model.predict(testX, verbose=verbose)
        if save_pred is not None:
            np.save(save_pred, yhat_class)
        if testY is not None and len(testY) == 2:
            acc = self.model.evaluate(testX, testY, verbose=verbose)
            if verbose:
                print("The classifier and discriminator metrics for evaluation are [{}, {}]".format(acc[0], acc[1]))
        elif testY is not None and len(testY) == 1:
            acc = self.model.evaluate(testX, [np.ones((testY.shape[0], 2)), testY], verbose=verbose)
            if verbose:
                print("The classifier metric for evaluation is {}".format(acc[1]))


In [ ]:
import keras
import numpy as np
import pickle as pkl

# LOAD DATASET
print("Load dataset")
f = gzip.open('pkl/sem-relations.pkl.gz', 'rb')
data = pkl.load(f)
f.close()

position_dims = 50

embeddings = data['wordEmbeddings']
# naming rule 
yTrain, sentenceTrain, position1Train, position2Train = data['train_set']
yTest, sentenceTest, position1Test, position2Test = data['test_set']
XTrain = [sentenceTrain, position1Train, position2Train]
XTest  = [sentenceTest, position1Test, position2Test]

# out of domain data, temporary use same data as above
temp, sentenceTrainDX, position1TrainDX, position2TrainDX = data['train_set']
temp, sentenceTestDX, position1TestDX, position2TestDX = data['test_set']

DXTrain = [sentenceTrainDX, position1TrainDX, position2TrainDX]
DXTest = [sentenceTestDX, position1TestDX, position2TestDX]

max_position = max(np.max(position1Train), np.max(position2Train)) + 1

n_out = max(yTrain) + 1

max_sentence_len = sentenceTrain.shape[1]

print("sentenceTrain : ", sentenceTrain.shape)
print("positionTrain1 : ", position1Train.shape)
print("positionTrain2 : ", position2Train.shape)

print("sentenceTest : ", sentenceTest.shape)
print("positionTest1 : ", position1Test.shape)
print("positionTest2 : ", position2Test.shape)


# Process data to match the model specs.
print("333333333333333")
trainX = np.expand_dims(sentenceTrain, axis=2)
trainX = np.tile(trainX, [1, 1, 2]).transpose(0, 1, 2)
trainY = yTrain

print(trainX.shape)

trainX = np.expand_dims(trainX, axis=3)
trainX = np.tile(trainX, [1, 1, 1, 1]).transpose(0, 3, 1, 2)
print(trainX.shape)

# # Divide train and test sets (the only limitation of this model is to assign even number as batch_sizes and the size of data should yield 0 remainder.)
# trainX, testX = trainX[:45000], trainX[45000:55016]
# trainY = keras.utils.to_categorical(trainY, num_classes=10)
# trainY, testY = trainY[:45000], trainY[45000:55016]

# Load MNIST-M dataset as out-of-domain and unlabeled data     
trainDX = trainX[:6000]
testDX = trainX[6000:8000]
mnist_m = None

# # Rescale -1 to 1
# trainX = (trainX.astype(np.float32) - 127.5) / 127.5
# trainDX = (trainDX.astype(np.float32) - 127.5) / 127.5
# testX = (testX.astype(np.float32) - 127.5) / 127.5
# testDX = (testDX.astype(np.float32) - 127.5) / 127.5

# Initiate the model
dann = DANN(summary=True, width=max_sentence_len, channels=1, classes=10, features=32, batch_size=32, model_plot=True)
# Train the model
dann.train(trainX, trainDX, trainY, epochs=100, batch_size=32)
# Evaluate for binary MNIST
dann.evaluate(testX, testY, save_pred="./binary_testX.npy", verbose=True)
# Evaluate for colorful MNIST (MNIST-M)
dann.evaluate(testDX, save_pred="./colorful_testX.npy", verbose=True)
